## PyCon UK '23 workshop: Turn Satellite Images into Digital Art with Python!
---

In [ ]:
%config Completer.use_jedi = False

In [ ]:
%pip install -q scipy

In [ ]:
%pip install -q matplotlib

In [ ]:
from time import time

In [ ]:
from PIL import Image

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

In [ ]:
# Load jpg 
raw_jpg = Image.open("img.jpg").convert("RGB")

In [ ]:
# Convert to numpy array
array_uint8 = np.array(raw_jpg) 

In [ ]:
array_uint8

In [ ]:
array_uint8.shape

In [ ]:
# Make function that generates a colour swatch of a specific image pixel
def pixel_colour_swatch(row, column):
    RGB_colour = tuple(array_uint8[row][column].tolist())
    print(f"The pixel at ({row}, {column}) has RGB of {RGB_colour}.")
    return Image.new("RGB", (50,50), RGB_colour)

In [ ]:
pixel_colour_swatch(10, 10)

In [ ]:
# Assign the dimensions of the array_uint8 object to variables
width, height, depth = original_shape = array_uint8.shape
original_shape

In [ ]:
# Test the condition that the value of the depth variable is 3 (as would be the case of the image colour mode is RGB)
# Reshape the array_uint8 object into basically width * height no. of rows and depth no. of columns (3)
assert depth == 3
reshaped_array_uint8 = np.reshape(array_uint8, (width * height, depth))

In [ ]:
# Check the dimensions
reshaped_array_uint8.shape

In [ ]:
reshaped_array_uint8[:5]

In [ ]:
# Calculate the number of colours in the original jpg
original_n_colors = np.unique(reshaped_array_uint8, axis=0).shape[0]
original_n_colors

In [ ]:
reshaped_array_uint8

In [ ]:
# Convert to the default 8 bits integer coding to floats and divide by 255 so the floats are in the range [0-1] (normalised)
reshaped_array_float01range = np.array(reshaped_array_uint8, dtype=np.float64) / 255

In [ ]:
reshaped_array_float01range.shape

In [ ]:
image_array = reshaped_array_float01range.copy()

In [ ]:
# Choose the number of clusters, i.e. colours, to reduce the original jpg down to
n_colors = 6

In [ ]:
# Train K-Means clustering model on small sub-sample of the data
print("Fitting model on a small sub-sample of the data")
t0 = time()
image_array_sample = shuffle(image_array, random_state=0, n_samples=1_000)
kmeans = KMeans(n_clusters=n_colors, n_init="auto", random_state=0).fit(
    image_array_sample
)
print(f"done in {time() - t0:0.3f}s.")

In [ ]:
# Get labels for all points | Use the kmeans trained model to predict which of the n clusters each pixel belongs to, i.e. assign each pixel a label, from 1 to n_colors
print("Predicting color indices on the full image (k-means)")
t0 = time()
labels = kmeans.predict(image_array)
print(f"done in {time() - t0:0.3f}s.")

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
# Check out available 'named colors' and what their RGB value is: https://matplotlib.org/stable/gallery/color/named_colors.html#css-colors
mcolors.to_rgb("cornflowerblue")

In [ ]:
# Write function for creating custom colour palettes, referred to here as a 'codebook'
# FYI can also use a list comprehension for the main body of the function: `codebook = np.array([mcolors.to_rgb(i) for i in args])`

def make_codebook(*args):
    codebook = []
    for i in args:
        codebook.append(mcolors.to_rgb(i))
    codebook = np.array(codebook)
    return codebook

In [ ]:
# Use the `make_codebook` function to create a custom colour palette with which to recreate an updated jpg with, i.e a new piece of Digital Art
# MAKE SURE you pass the same number of 'named colors' as 'n_colors', e.g. 6

codebook_6pastels = make_codebook("mediumspringgreen", "cornflowerblue", "thistle", "coral", "khaki",  "darkturquoise")

In [ ]:
labels

In [ ]:
codebook_6pastels

In [ ]:
# Write function to recreate the (compressed) image from an inputted codebook, pixel labels, image width, and image height

def recreate_image(codebook, labels, width, height):
    return codebook[labels].reshape(width, height, -1)

In [ ]:
# OPTIONAL: use this code (uncommented!) to see what the `codebook[labels]` code in the 'recreate_inage' function is doing
# test_codebook = make_codebook("orange", "navy")
# test_labels = np.array([0,1,1,0,1,0,0])
# test_codebook[test_labels]

In [ ]:
# Create and plot `SatArt` variable - the recreated image reduced to n_colors and with custom codebook colour palette
SatArt = recreate_image(codebook_6pastels, labels, width, height)
plt.figure(figsize=(10,10))
plt.clf()
plt.axis("off")
plt.title(f"Satellite Art - a quantized image with {n_colors} colours")
plt.imshow(SatArt);
#plt.savefig("SatArt_test.png", dpi=600)

In [ ]:
from PIL.ImageFilter import (FIND_EDGES, CONTOUR, EMBOSS, MedianFilter, MinFilter, MaxFilter)

In [ ]:
from PIL.ImageOps import solarize

In [ ]:
# Convert `SatArt` into RGB image, scaling the 0-1 values and casting/conversion back to uint8/8 bit integer format
SatArt_uint8 = Image.fromarray((SatArt * 255).astype(np.uint8))

In [ ]:
# Check out some ready-made filters from the Python Imaging Library (PIL)

PIL_filters = ["FIND_EDGES", "FIND_EDGES (greyscale)", "CONTOUR", "CONTOUR (greyscale)", "EMBOSS", "MedianFilter", "MinFilter", "MaxFilter",  "solarize"]
fig, ax = plt.subplots(9,1, figsize=(40,80))

ax[0].imshow(SatArt_uint8.filter(FIND_EDGES()))    
ax[1].imshow(SatArt_uint8.convert("L").filter(FIND_EDGES()))    # Can also try converting the image to greyscale before filtering
ax[2].imshow(SatArt_uint8.filter(CONTOUR()))     
ax[3].imshow( SatArt_uint8.convert("L").filter(CONTOUR()))    # Can also try converting the image to greyscale first before filtering

ax[4].imshow(SatArt_uint8.filter(EMBOSS()))
ax[5].imshow(SatArt_uint8.filter(MedianFilter(size=5)))
ax[6].imshow(SatArt_uint8.filter(MinFilter(size=5)))
ax[7].imshow(SatArt_uint8.filter(MaxFilter(size=5)))
ax[8].imshow(solarize(SatArt_uint8, threshold=0)) # Threshold can range from 0-128

for count, value in enumerate(PIL_filters):
    ax[count].axis("off")
    ax[count].set_title(f"{count+1}. Quantized image with {value} PIL filter")
plt.show()

In [ ]:
# Chain the `.save()` function to the expression with a string input such as "Egypt.png" 
# SatArt_uint8.convert("L").filter(FIND_EDGES()).save("...png")

In [ ]:
# Before vs After side-by-side comparison

fig, ax = plt.subplots(1,2, figsize=(40,30))
ax[0].imshow(raw_jpg)
ax[1].imshow(SatArt_uint8.convert("L").filter(FIND_EDGES()))
ax[0].axis("off")
ax[1].axis("off")

ax[0].set_title(f"Original Satellite screenshot with {original_n_colors} colours")
ax[1].set_title(f"Digital Satellite Art")
plt.show()

---
Copyright © 2023 Rho Zeta AI Ltd. All rights reserved.